In [1]:
from teadata import DataEngine
from teadata.classes import haversine_miles
import pandas as pd
from pprint import pprint

# Enter the district name or district number of the school district you wish to analyze
DISTRICT_NAME = "Austin ISD"

# Instantiate DataEngine object, a layer over the District and Campus objects
repo = DataEngine.from_snapshot(search=True)

# Inspect how many objects are loaded
print(f"Loaded {len(repo.districts)} districts/charters and {len(repo.campuses)} campuses")

Loaded 1207 districts/charters and 9739 campuses


In [2]:
# Select a district using either the district name (case insensitive) or district number (any format works - integer and string with or without a leading apostrophe are both acceptable)
district = (repo >> ("district", DISTRICT_NAME)).first()
print("Example district:", district.name, district.district_number)

# Access canonical and enriched attributes (dot-syntax works if enrichment logic is wired correctly)
print("\n2025 overall accountability rating:", district.rating, district.overall_rating_2025)

# Explore campuses inside the district
print(f"\n{district.name} has {len(district.campuses)} total campuses.")
print(f"\n{district.name} campus ratings:", district.campuses.value_counts("rating"))
print(f"\n{district.name} school types:", district.campuses.value_counts("school_type"))

Example district: Austin ISD '227901

2025 overall accountability rating: C C

Austin ISD has 127 total campuses.

Austin ISD campus ratings: [('C', 27), ('B', 25), ('F', 23), ('A', 22), ('D', 18), ('Not Rated', 7), ('Not Rated: Minimum Size rules', 1)]

Austin ISD school types: [('Elementary School', 81), ('High School', 17), ('Middle School', 17), ('Elementary/Secondary', 10), ('Junior High School', 2)]


In [3]:
# Select all of a district's campuses, then filter and sort and take the top 5 by enrollment, then find the nearest charter school to each campus with the same "school type" ("Elementary School", "Junior High School", "Middle School", "Elementary/Secondary", "High School")

# TODO: Is a better comparison to find overlapping grades served?

top5 = (
    repo
    >> ("campuses_in", district)
    >> ("where", lambda c: (c.enrollment or 0) > 0)
    >> ("sort",  lambda c: c.enrollment or 0, True)
    >> ("take", 5)
    >> ("nearest_charter_same_type",)   # yields list of (campus, match, miles)
)

In [4]:
# Each data type includes .to_dict() (or .to_dicts() if a list) and .to_df() helper functions for exporting or working with query results
df = top5.to_df()  # auto-flattened & prefixed

# Define the columns we are interested in and the order we want
cols = [
    "campus_name", "campus_enrollment", "campus_rating", "campus_school_type",
    "match_name", "match_enrollment", "match_rating", "match_school_type",
    "distance_miles"
]

# Apply filter/sort to dataframe
df = df[cols]

# Display dataframe
df

,campus_name,campus_enrollment,campus_rating,campus_school_type,match_name,match_enrollment,match_rating,match_school_type,distance_miles
0,BOWIE H S,2876,A,High School,KIPP AUSTIN BRAVE,490,B,High School,6.143997
1,AKINS EARLY COLLEGE H S,2421,C,High School,KIPP AUSTIN BRAVE,490,B,High School,4.610669
2,AUSTIN H S,2323,B,High School,TEXANS CAN ACADEMY - AUSTIN,168,C,High School,3.464640
3,ANDERSON H S,2223,B,High School,CEDARS ACADEMY NEXT GENERATION H S AT HIGHLAND,116,A,High School,4.132389
4,MCCALLUM H S,1897,A,High School,CEDARS ACADEMY NEXT GENERATION H S AT HIGHLAND,116,A,High School,1.352387


In [5]:
# Get charter campuses physically inside that district’s boundary - either syntax works
charters = repo.charter_campuses_within(district)
charters = repo >> ("charters_within", district)

print(f"{district.name}: {len(charters)} charter campuses inside boundary")

Austin ISD: 38 charter campuses inside boundary


In [6]:
# Similar to before, export the result of our query to a dataframe for display after filtering and sorting the columns
df = charters.to_df()

cols = [
    "name", "campus_number", "district_number", "charter_type", "aea", "enrollment", "overall_rating_2025", "grade_range", "school_type", "school_status_date", "update_date",
]

df = df[cols]

df

,name,campus_number,district_number,charter_type,aea,enrollment,overall_rating_2025,grade_range,school_type,school_status_date,update_date
0,BASIS AUSTIN,'015834004,'015834,OPEN ENROLLMENT CHARTER,N,155,A,'03-12,Elementary/Secondary,2021-08-01,2025-03-11
1,BASIS AUSTIN PRI,'015834102,'015834,OPEN ENROLLMENT CHARTER,N,243,Not Rated: Minimum Size rules,'KG-02,Elementary School,2020-10-12,2025-03-11
2,TEXANS CAN ACADEMY - AUSTIN,'057804006,'057804,OPEN ENROLLMENT CHARTER,Y,168,C,'09-12,High School,2013-06-28,2025-03-11
3,PREMIER HIGH SCHOOL-AUSTIN SOUTH CAMPUS,'072801113,'072801,OPEN ENROLLMENT CHARTER,Y,143,A,'06-12,Elementary/Secondary,2011-05-31,2025-03-11
4,TEXAS PREPARATORY SCHOOL - AUSTIN CAMPUS,'105802101,'105802,OPEN ENROLLMENT CHARTER,N,59,F,'KG-06,Elementary School,2013-08-26,2025-03-11
5,KI CHARTER - AUSTIN,'105803007,'105803,OPEN ENROLLMENT CHARTER,N,-1,Not Rated: Minimum Size rules,'01-12,Elementary/Secondary,2024-08-16,2025-03-11
6,IDEA RUNDBERG COLLEGE PREPARATORY,'108807036,'108807,OPEN ENROLLMENT CHARTER,N,790,A,'06-12,Elementary/Secondary,2015-07-15,2025-03-11
7,IDEA BLUFF SPRINGS COLLEGE PREPARATORY,'108807037,'108807,OPEN ENROLLMENT CHARTER,N,689,B,'06-12,Elementary/Secondary,2016-08-09,2025-03-11
8,IDEA HEALTH PROFESSIONS COLLEGE PREPARATORY,'108807091,'108807,OPEN ENROLLMENT CHARTER,N,376,B,'06-11,Elementary/Secondary,2019-08-06,2025-03-11
9,IDEA BLUFF SPRINGS ACADEMY,'108807137,'108807,OPEN ENROLLMENT CHARTER,N,521,D,'KG-05,Elementary School,2016-08-09,2025-03-11
